In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import warnings

warnings.filterwarnings(action='ignore')
rc('font',family='AppleGothic')
rcParams['axes.unicode_minus'] = False

In [2]:
path = '../data/'

train = pd.read_csv(path+"3_train.csv")
# train_errx = pd.read_csv(path+"3_train_errx.csv")
test = pd.read_csv(path+"3_test.csv")
sub = pd.read_csv(path + 'sample_submission.csv')

# train.shape, train_errx.shape, test.shape
train.shape, test.shape

((2952, 67), (1008, 66))

In [3]:
print(train.iloc[:,:14].isnull().sum())
print(test.iloc[:,:14].isnull().sum())

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금         0
임대료           0
지하철         211
버스정류장         0
단지내주차면수       0
dtype: int64
단지코드         0
총세대수         0
임대건물구분       0
지역           0
공급유형         0
전용면적         0
전용면적별세대수     0
공가수          0
자격유형         0
임대보증금        0
임대료          0
지하철         38
버스정류장        0
단지내주차면수      0
dtype: int64


In [4]:
def create_new_X(old):
    # 전용면적의 소수점 없애기
    old['전용면적'] = old['전용면적'].apply(int)
    
    # 겹치는 전용면족 행 병합(결측치 없는 피처로 구분)
    cols = list(old.columns)
    cols.remove('전용면적별세대수')
    cols.remove('지하철')
    by_area = old.groupby(cols, as_index=False).agg({'전용면적별세대수': 'sum'})
    
    # 각 전용면적과 그에 상응하는 세대수의 곱
    by_area['면적_세대수_곱']  = by_area['전용면적'] * by_area['전용면적별세대수']
    
    # 각 단지코드당 계산된 실제 총 세대수
    by_area['면적_세대수_곱_총합'] = by_area['면적_세대수_곱'].groupby(by_area['단지코드']).transform('sum')
    
    # 비율 계산
    by_area['면적_세대수_비율'] = by_area['면적_세대수_곱']/by_area['면적_세대수_곱_총합']
    
    return by_area

new_train = create_new_X(train)
print(train.shape, "→", new_train.shape)
new_train.head()

(2952, 67) → (1547, 69)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,O,ERROR1_lb,나머지,분양상가,분양아파트,등록차량수,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합,면적_세대수_비율
0,C1000,566,아파트,충청북도,국민임대,39,10.0,A,10803000.0,96030.0,...,0,0,1,0,0,481.0,419,16341,23478,0.696013
1,C1000,566,아파트,충청북도,국민임대,46,10.0,A,14404000.0,129640.0,...,0,0,1,0,0,481.0,72,3312,23478,0.141068
2,C1000,566,아파트,충청북도,국민임대,51,10.0,A,15604000.0,150040.0,...,0,0,1,0,0,481.0,75,3825,23478,0.162918
3,C1004,521,상가,충청남도,임대상가,14,3.0,D,10686510.0,94379.0,...,0,0,1,0,0,93.0,1,14,20315,0.000689
4,C1004,521,상가,충청남도,임대상가,19,3.0,D,10686510.0,74005.5,...,0,0,1,0,0,93.0,1,19,20315,0.000935


In [5]:
new_test = create_new_X(test)
print(new_test.columns)
new_test

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '버스정류장', '단지내주차면수', 'ERROR1', '임대건물구분_lb', '상가', '아파트', '지역_lb',
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공급유형_lb', '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)',
       '국민임대', '영구임대', '임대상가', '장기전세', '행복주택', '자격유형_lb', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'ERROR1_lb',
       '나머지', '분양상가', '분양아파트', '전용면적별세대수', '면적_세대수_곱', '면적_세대수_곱_총합',
       '면적_세대수_비율'],
      dtype='object')


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,N,O,ERROR1_lb,나머지,분양상가,분양아파트,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합,면적_세대수_비율
0,C1003,480,아파트,경상남도,행복주택,16,29.0,J,12000000.0,61000.0,...,0,0,0,1,0,0,128,2048,12492,0.163945
1,C1003,480,아파트,경상남도,행복주택,26,29.0,J,18800000.0,96000.0,...,0,0,0,1,0,0,30,780,12492,0.062440
2,C1003,480,아파트,경상남도,행복주택,26,29.0,J,19600000.0,100000.0,...,0,0,0,1,0,0,220,5720,12492,0.457893
3,C1003,480,아파트,경상남도,행복주택,36,29.0,J,25600000.0,131000.0,...,0,0,0,1,0,0,68,2448,12492,0.195965
4,C1003,480,아파트,경상남도,행복주택,44,29.0,J,30000000.0,154000.0,...,0,0,0,1,0,0,34,1496,12492,0.119757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,C2688,150,아파트,전라남도,국민임대,46,1.0,H,16448000.0,194290.0,...,0,0,0,1,0,0,16,736,4552,0.161687
563,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9113000.0,86450.0,...,0,0,0,1,0,0,26,624,4552,0.137083
564,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9163000.0,86940.0,...,0,0,0,1,0,0,4,96,4552,0.021090
565,C2691,765,아파트,경기도,공공임대(10년),74,25.0,A,66000000.0,550000.0,...,0,0,0,1,0,0,175,12950,62510,0.207167


In [6]:
# 여기 공식이 바뀔수있어야한다.
def create_new_y(new_X):
    new_X['전용면적_비례_차량수'] = new_X.apply(lambda row: (row['면적_세대수_곱']) / row['면적_세대수_곱_총합'] * row['등록차량수'], axis=1)
    return new_X['전용면적_비례_차량수']

new_y = create_new_y(new_train)
new_y


0       334.782392
1        67.853821
2        78.363787
3         0.064091
4         0.086980
           ...    
1542    154.982603
1543    108.086386
1544    759.160801
1545    178.780726
1546    167.972087
Name: 전용면적_비례_차량수, Length: 1547, dtype: float64

In [7]:
def get_complex_code(new_X):
    return new_X['단지코드']

In [8]:
def revert_to_old_y(new_y, new_complex_code):
    return pd.concat([new_complex_code, new_y], axis=1).groupby("단지코드").sum()
    

In [9]:
# train으로 revert잘되는지 테스트
new_complex_code = get_complex_code(new_train)
old_y = revert_to_old_y(new_y, new_complex_code)
old_y
# 훈련을 시킨다
# test_X -> create_new_X로 변환시킨다.
# 모델에 predict 시키면 -> test_new_y가 나온다.


,전용면적_비례_차량수
단지코드,
C1000,481.0
C1004,93.0
C1005,376.0
C1013,1665.0
C1014,708.0
...,...
ERR04_2051,780.0
ERR04_2483,1015.0
N1036,1279.0


## 김동화

In [10]:
new_train['전용면적_비례_차량수'] = new_y
print(new_train.shape)
print(new_train.columns)
new_train.head()

(1547, 70)
Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '버스정류장', '단지내주차면수', 'ERROR1', '임대건물구분_lb', '상가', '아파트', '지역_lb',
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공급유형_lb', '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)',
       '국민임대', '영구임대', '임대상가', '장기전세', '행복주택', '자격유형_lb', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'ERROR1_lb',
       '나머지', '분양상가', '분양아파트', '등록차량수', '전용면적별세대수', '면적_세대수_곱', '면적_세대수_곱_총합',
       '면적_세대수_비율', '전용면적_비례_차량수'],
      dtype='object')


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,ERROR1_lb,나머지,분양상가,분양아파트,등록차량수,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합,면적_세대수_비율,전용면적_비례_차량수
0,C1000,566,아파트,충청북도,국민임대,39,10.0,A,10803000.0,96030.0,...,0,1,0,0,481.0,419,16341,23478,0.696013,334.782392
1,C1000,566,아파트,충청북도,국민임대,46,10.0,A,14404000.0,129640.0,...,0,1,0,0,481.0,72,3312,23478,0.141068,67.853821
2,C1000,566,아파트,충청북도,국민임대,51,10.0,A,15604000.0,150040.0,...,0,1,0,0,481.0,75,3825,23478,0.162918,78.363787
3,C1004,521,상가,충청남도,임대상가,14,3.0,D,10686510.0,94379.0,...,0,1,0,0,93.0,1,14,20315,0.000689,0.064091
4,C1004,521,상가,충청남도,임대상가,19,3.0,D,10686510.0,74005.5,...,0,1,0,0,93.0,1,19,20315,0.000935,0.086980


In [11]:
print(new_test.shape)
print(new_test.columns)
new_test

(567, 68)
Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수', '자격유형', '임대보증금',
       '임대료', '버스정류장', '단지내주차면수', 'ERROR1', '임대건물구분_lb', '상가', '아파트', '지역_lb',
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공급유형_lb', '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)',
       '국민임대', '영구임대', '임대상가', '장기전세', '행복주택', '자격유형_lb', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'ERROR1_lb',
       '나머지', '분양상가', '분양아파트', '전용면적별세대수', '면적_세대수_곱', '면적_세대수_곱_총합',
       '면적_세대수_비율'],
      dtype='object')


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,N,O,ERROR1_lb,나머지,분양상가,분양아파트,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합,면적_세대수_비율
0,C1003,480,아파트,경상남도,행복주택,16,29.0,J,12000000.0,61000.0,...,0,0,0,1,0,0,128,2048,12492,0.163945
1,C1003,480,아파트,경상남도,행복주택,26,29.0,J,18800000.0,96000.0,...,0,0,0,1,0,0,30,780,12492,0.062440
2,C1003,480,아파트,경상남도,행복주택,26,29.0,J,19600000.0,100000.0,...,0,0,0,1,0,0,220,5720,12492,0.457893
3,C1003,480,아파트,경상남도,행복주택,36,29.0,J,25600000.0,131000.0,...,0,0,0,1,0,0,68,2448,12492,0.195965
4,C1003,480,아파트,경상남도,행복주택,44,29.0,J,30000000.0,154000.0,...,0,0,0,1,0,0,34,1496,12492,0.119757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,C2688,150,아파트,전라남도,국민임대,46,1.0,H,16448000.0,194290.0,...,0,0,0,1,0,0,16,736,4552,0.161687
563,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9113000.0,86450.0,...,0,0,0,1,0,0,26,624,4552,0.137083
564,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9163000.0,86940.0,...,0,0,0,1,0,0,4,96,4552,0.021090
565,C2691,765,아파트,경기도,공공임대(10년),74,25.0,A,66000000.0,550000.0,...,0,0,0,1,0,0,175,12950,62510,0.207167


### csv, xlsx 파일로 내보내기

In [12]:
path = './data_전용면적별/'

new_train.to_csv(path + "train_new.csv", index=False)
new_test.to_csv(path + "test_new.csv", index=False)

new_train.to_excel(path + "train_new.xlsx", index=False)
new_test.to_excel(path + "test_new.xlsx", index=False)

os.listdir(path)

['train_new.csv', 'test_new.csv', 'train_new.xlsx', 'test_new.xlsx']

In [13]:
# 2021.07.26 update 누락된 단지코드 있는지 확인
tr_codes = set(train['단지코드'].unique())
ts_codes = set(test['단지코드'].unique())

tr_new_codes = set(new_train['단지코드'].unique())
ts_new_codes = set(new_test['단지코드'].unique())

tr_miss = tr_codes - tr_new_codes
ts_miss = ts_codes - ts_new_codes

print(tr_miss)
print(ts_miss)

set()
set()
